<a href="https://colab.research.google.com/github/j54854/myColab/blob/main/pom2_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 生産管理技術2_10の補助資料

この補助資料では，生産管理技術2の第10回講義で扱った単一機械での見込み生産における在庫管理方策の事例で，方策パラメータのブラックボックス最適化ために用いた離散時間シミュレータと粒子群最適化（Particle Swarm Optimization: PSO）について簡単に説明する．

## 想定する在庫管理問題

見込み生産では，市場からの需要に在庫で応える．ここに，t期の需要 $d_t$ は次式で与えられるものとしよう．

$d_t$ = BASE + CV $\times d_{t-1} +\epsilon_t$

ただし，BASEとCVは所与のパラメータであり，$\epsilon_t$ は，tにかかわらず独立に，平均0，標準偏差SDの正規分布に従う確率変数であるとする．このとき，定常状態での1期あたりの需要の平均は

MEAN = BASE /(1 -CV)

となる．さらに，発注には1回あたりC_Oの費用がかかり，発注から納入までのリードタイムは
LT，手元在庫の保管費用は1個1期あたりC_H，欠品には1個1期あたりC_Sのペナルティがかかるとする．

この在庫の管理には，(s, S)方策が利用されている．すなわち，毎期末に（発注済み未入荷分やバックオーダーを考慮した正味）在庫量を確認し，それが発注点s未満であれば（その場合に限り），補充点Sとの差を発注する．

問題は，発注費用，保管費用，欠品費用を合わせた１期あたりの平均コストを最小にするように，在庫保管方策のパラメータ（sとS）の値を決めることである．ただし，この平均コストを解析的に求めるのは簡単ではないため，ここでは，シミュレーションによって評価することにする．そうすると，これは典型的なブラックボックス最適化問題となる．

## 離散時間シミュレータ

最初に，上の情報を格納しておくための大域変数を用意し，シミュレータで利用するライブラリ（random，math）を読み込んでおく．なお，初出のHZNは，シミュレーションを続ける期間の長さ（タイムホライズン），DRYは，そのうち最初の何期分のデータを分析の対象外（初期値に左右される外れ値）として除外するか，を表している．

In [ ]:
import math, random

HZN, DRY = 150, 50  # simulation time horizon (first DRY periods are dry run)
BASE, SD, CV = 30, 10, 0.8  # parameters of demand time series
MEAN = round(BASE /(1 -CV))  # mean of demand time series
LT = 3  # lead time
C_O, C_H, C_S = 1000, 1, 10  # ordering, holding, and shortage costs

シミュレーションモデルのクラスは下記のようになっている．

まず，フィールドのsとSは，在庫管理方策のパラメータであるが，後で行うブラックボックス最適化の際に解空間を整数に限定しなくてもよいようにするため，引数で受け取った値を整数に丸める処理が加えられている．on_handは手元在庫量，back_orderはバックオーダー，dは需要量，orderは発注済み未入荷オーダーのリストである．

get_demand()メソッドは上の需要のモデル式に対応しており，これを呼ぶたびに，その次の期の需要量が返される．また，s_total()メソッドは，発注済み未入荷分やバックオーダーを考慮した正味在庫量を計算するものである．

receive()は，納入時の処理を表すメソッド，ship_out()は，出荷時の処理を表すメソッド，place_order()は，発注の意思決定を表現したメソッド，にそれぞれなっている．

In [ ]:
class Model:
    def __init__(self, s, S):
        self.s, self.S = round(s), round(S)  # parameters of inventory control policy
        self.on_hand = self.S  # stock on hand
        self.back_order = 0  # back order
        self.d = MEAN  # last period demand
        self.order = [0 for _ in range(LT)]  # list of unfulfilled orders

    def get_demand(self):  # set new demand based on last period demand
        self.d = round(BASE + CV *self.d +random.normalvariate(0, SD))

    def s_total(self):  # level of total available stock
        return self.on_hand +sum(self.order) -self.back_order

    def receive(self):  # fulfill order placed at t -LT
        self.on_hand += self.order.pop(0)

    def ship_out(self):
        gross = self.d +self.back_order  # gross demand including back order
        flow = min(gross, self.on_hand)  # how many to ship out
        self.on_hand -= flow  # take out flow from stock on hand
        self.back_order = gross -flow  # if flow < gross, back order remains

    def place_order(self):
        if self.s_total() < self.s:  # only when available stock is lower than s
            self.order.append(self.S -self.s_total())  # recover stock level to S
            return True
        else:  # otherwise
            self.order.append(0)  # place no order
            return False

    def print_state(self, t, before, after):
        print('[t={}] '.format(t), end='')
        print('stock:{} ({}+{}-{})'.format(*before), end=' -> demand:')
        print(self.d, end=' -> ')
        print('stock:{} ({}+{}-{}) -> order:'.format(*after), end='')
        print(self.order[-1])

    def run(self, display=True):
        cost = 0
        for t in range(HZN):
            self.receive()
            self.get_demand()
            before_shipment = (self.s_total(), self.on_hand, sum(self.order), self.back_order, self.d)
            self.ship_out()
            is_ordered = self.place_order()
            after_shipment = (self.s_total(), self.on_hand, sum(self.order), self.back_order)
            if display:
                self.print_state(t, before_shipment, after_shipment)
            if t >= DRY:  # if dry run has been completed
                mean_on_hand = (before_shipment[1] +after_shipment[1]) /2
                cost += is_ordered *C_O +mean_on_hand *C_H +self.back_order *C_S
        return cost /(HZN -DRY)

最後のrun()がシミュレーションを実行するメソッドである．ここに，離散時間シミュレーションの流れが凝縮されている．時間をt=0からHZN-1まで1ずつインクリメントしていくforrループがあり，その中で，毎度，期中の処理が順に呼ばれている．引数のdisplayは，シミュレーション中に各期の状態を画面に打ち出すかどうかを指定するためのものであり，もしそれがTrueであれば，print_state()メソッドを呼んでいる．

### 実行方法

作成したシミュレータを動かすには，単に，モデルクラスをインスタンス化して，そのrun()メソッドを呼べばよい．準備の段階で，

HZN = 150

としているので，150期分の状態が順に画面に流れていく．

実行前に大域変数の値を変更することで，タイムホライズンだけでなく，需要やリードタイム，コストの条件を変えてみることもできる（ので試してみてほしい）．

randomモジュールで生成される疑似乱数を使用しているので，実行するたびに結果が異なることになるが，同じシード値を指定してrandom.seed()を呼んでおくと，同一の乱数系列を使用（して，同じ結果を再現）することができる．

In [ ]:
# random.seed(999)  # if seed is specified, the same random number are used

stock = Model(500, 1000)
cost = stock.run()
print('cost = {}'.format(cost))

## ブラックボックス最適化

続いて，平均コストの最小化を目指して，方策パラメータのブラックボックス最適化を試みよう．ここでは，比較的単純なアルゴリズムである程度良い結果が得られやすい粒子群最適化（Particle Swarm Optimization: PSO）を利用してみよう．

最初に，pipで，PSOのためのライブラリ（pyswarms）をインストールする．

In [ ]:
!pip install pyswarms

次に，下記のように，このpyswarmsに加え，numpyとmatplotlibを読み込んでおく（シミュレータにはあえて利用していなかったが，pysmarmsにはnumpyが必要らしい）．

In [ ]:
import numpy as np
import pyswarms as ps

import matplotlib.pyplot as plt
from pyswarms.utils.plotters import plot_cost_history

続いて，ブラックボックス最適化の目的関数をfx()として定義しておこう．下のように，引数をX，返り値をYとしている．

ここで，pyswarmsを利用するためには，引数Xは，複数の決定変数ベクトルをまとめた2次元のnumpy配列に，返り値Yは，個々の決定変数ベクトルに対応する目的関数値を並べた1次元のnumpy配列に，それぞれしておく必要がある．

関数の中身を見ると，指定された各方策パラメータ値のもとで上のシミュレーションを実行し，得られたコストのリストをnumpy配列にして返しているだけである．

ここで，(s, S)そのものではなく，(s/1000, S/2000)を決定変数xとしている点に注意してほしい．これは，2つの決定変数の探索範囲をどちらもだいたい[0, 1]の範囲に正規化するためである．また，平均コストを評価するためのサンプル値を増やすためにHZNの値を大きくし，毎度同じ乱数系列を使用できるように新たな大域変数SEEDを導入している．

In [ ]:
HZN = 5050  # simulation horizon for PSO
SEED = 999  # random seed for PSO

def fx(X):
    Y = []
    for x in X:
        random.seed(SEED)
        stock = Model(x[0] *1000, x[1] *2000)
        y = stock.run(display=False)
        Y.append(y)
    return np.array(Y)

さて，これで準備が整ったので．pyswarmsを利用して，方策パラメータの最適化を試みよう．optionsは，PSOのハイパーパラメータ（の一部）である．ここでは，ライブラリの推奨値をそのまま使っている．さらに，下では，粒子数10，反復数1000に設定している．

In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options)
cost, pos = optimizer.optimize(fx, iters=1000)
print('----- (semi)optimal solution found -----')
print('(s, S)^* = ({}, {}) and cost = {}'.format(round(pos[0] *1000), round(pos[1] *2000), cost))

このように，比較的簡単に解を得ることができる（ので，条件を変更して試してみてほしい）．また，最適化の過程でコストが削減されていく様子は，下のコードで確認することができる．

In [ ]:
plot_cost_history(cost_history=optimizer.cost_history)
plt.show()